# J001 Assignment 4

## Task 1: Query Matching System

In [21]:
import pandas as pd, re, warnings
from rapidfuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')

In [ ]:
query_data_new = pd.read_csv("data/new_queries.csv")
query_data_resolved = pd.read_csv("data/resolved_queries.csv")
new_queries, resolved_queries = query_data_new, query_data_resolved

In [23]:
new_queries, resolved_queries

(                              Variation_Query  Matches_With_Query_ID
 0            Unabel to conect to the internet                      1
 1                   Can’t connect to internet                      1
 2                         Intenet not working                      1
 3                Payment failed while chekout                      2
 4   Payment did not go through during chckout                      2
 5                  Payment issue at check out                      2
 6    Application crashes when opening setings                      3
 7            App crash when going to settings                      3
 8            Settings cause the app to chrash                      3
 9               Forgot passwrd and cant reset                      4
 10        Forgotten password, unable to reset                      4
 11                  I can’t reset my password                      4
 12             Unable to uplod file to server                      5
 13        Can't upl

### Text Preprocessing

In [24]:
def clean_text_data(input_text):
    if pd.isna(input_text) or input_text is None:
        return ""
    
    text = str(input_text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

new_queries["processed_text"] = new_queries["Variation_Query"].apply(clean_text_data)
resolved_queries["processed_text"] = resolved_queries["Pre_Resolved_Query"].apply(clean_text_data)

In [25]:
def find_fuzzy_match(query, ref_df, threshold=70):
    reference_texts = ref_df["Pre_Resolved_Query"].tolist()
    
    scoring_methods = {
        'ratio': fuzz.ratio,
        'partial': fuzz.partial_ratio,
        'token_sort': fuzz.token_sort_ratio,
        'token_set': fuzz.token_set_ratio
    }
    
    best_score = 0
    best_match = None
    best_method = None
    
    for method_name, scorer in scoring_methods.items():
        result = process.extractOne(query, reference_texts, scorer=scorer)
        if result and result[1] > best_score:
            best_score = result[1]
            best_match = result[0]
            best_method = method_name
    
    if best_score >= threshold:
        query_id = ref_df[ref_df["Pre_Resolved_Query"] == best_match]["Query_ID"].iloc[0]
        return best_method, best_match, query_id, best_score
    
    return None, None, None, None

fuzzy_results = []
for q in new_queries["Variation_Query"]:
    method, match, qid, score = find_fuzzy_match(q, resolved_queries)
    fuzzy_results.append((q, method, match, qid, score))

fuzzy_df = pd.DataFrame(fuzzy_results, columns=[
    "Query", "Method", "Match", "ID", "Score"
])

### TF-IDF and Cosine Similarity

In [26]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2), 
    max_features=5000,
    min_df=1,
    max_df=0.9
)

all_texts = new_queries["Variation_Query"].tolist() + resolved_queries["Pre_Resolved_Query"].tolist()
all_vectors = vectorizer.fit_transform(all_texts)

n_new = len(new_queries)
new_tfidf = all_vectors[:n_new]
resolved_tfidf = all_vectors[n_new:]

cosine_scores = cosine_similarity(new_tfidf, resolved_tfidf)

tfidf_results_list = []
for i, query_text in enumerate(new_queries["Variation_Query"]):
    best_idx = cosine_scores[i].argmax()
    similarity_score = cosine_scores[i][best_idx]
    matched_entry = resolved_queries.iloc[best_idx]
    tfidf_results_list.append((query_text, matched_entry["Pre_Resolved_Query"], matched_entry["Query_ID"], similarity_score))

tfidf_results = pd.DataFrame(tfidf_results_list, columns=[
    "Unresolved_Query", "TFIDF_Match", "TFIDF_Query_ID", "TFIDF_Score"
])

In [27]:
fuzzy_matches = [(row["Query"], row["Method"], row["Match"], row["ID"], row["Score"]) 
                 for _, row in fuzzy_df.iterrows()]

tfidf_matches = [(row["Unresolved_Query"], row["TFIDF_Match"], row["TFIDF_Query_ID"], row["TFIDF_Score"]) 
                 for _, row in tfidf_results.iterrows()]

fuzzy_clean = pd.DataFrame(fuzzy_matches, columns=[
    "Unresolved_Query", "Best_Method", "Fuzzy_Match", "Fuzzy_Query_ID", "Fuzzy_Score"
])

tfidf_clean = pd.DataFrame(tfidf_matches, columns=[
    "Unresolved_Query", "TFIDF_Match", "TFIDF_Query_ID", "TFIDF_Score"
])

combined = pd.merge(fuzzy_clean, tfidf_clean, on="Unresolved_Query", how="inner")

### Final Match Selection

In [28]:
def pick_final(row, fuzzy_thresh=75, tfidf_thresh=0.65):
    if row["Fuzzy_Score"] and row["Fuzzy_Score"] >= fuzzy_thresh:
        return row["Fuzzy_Match"], row["Fuzzy_Query_ID"], f"Fuzzy-{row['Best_Method']}"
    elif row["TFIDF_Score"] >= tfidf_thresh:
        return row["TFIDF_Match"], row["TFIDF_Query_ID"], "TFIDF"
    else:
        return None, None, "No Match"

combined[["Final_Match", "Final_Query_ID", "Method_Used"]] = combined.apply(
    pick_final, axis=1, result_type="expand"
)

combined[['Unresolved_Query', 'Fuzzy_Match', 'Method_Used']]

,Unresolved_Query,Fuzzy_Match,Method_Used
0,Unabel to conect to the internet,Unable to connect to the internet,Fuzzy-ratio
1,Can’t connect to internet,Unable to connect to the internet,Fuzzy-token_set
2,Intenet not working,None,No Match
3,Payment failed while chekout,Payment failed during checkout,Fuzzy-ratio
4,Payment did not go through during chckout,Payment failed during checkout,No Match
5,Payment issue at check out,Payment failed during checkout,No Match
6,Application crashes when opening setings,App crashes when opening settings,Fuzzy-partial
7,App crash when going to settings,App crashes when opening settings,Fuzzy-ratio
8,Settings cause the app to chrash,None,No Match
9,Forgot passwrd and cant reset,Forgot password and unable to reset,Fuzzy-ratio


## Task 2: Name Matching System

In [ ]:
name_variations = pd.read_csv("data/name_variations.csv")
base_names = pd.read_csv("data/base_names.csv")

base_names, name_variations

(    Base_Name_ID          Base_Name
 0              1         John Smith
 1              2     Jennifer Brown
 2              3   Michael O'Connor
 3              4       Maria Garcia
 4              5         Robert Lee
 5              6      Linda Johnson
 6              7      William Davis
 7              8   Elizabeth Wilson
 8              9     David Martinez
 9             10        Susan Clark
 10            11    James Rodriguez
 11            12         Mary Lewis
 12            13         Paul Allen
 13            14        Karen Young
 14            15        Thomas King
 15            16       Nancy Wright
 16            17       Daniel Scott
 17            18        Sandra Hill
 18            19  Christopher Green
 19            20      Jessica Adams,
           Variation Matches_With_Base_Name
 0      Thomas  King            Thomas King
 1        ThomasKing            Thomas King
 2      Maria Garcia           Maria Garcia
 3         MaryLewis             Mary Lewis
 4

In [30]:
def normalize_name(name: str) -> str:
    if pd.isna(name):
        return ""
    name = name.strip()
    name = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', name)
    name = name.lower()
    name = re.sub(r'[^a-z\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

name_variations["Normalized"] = name_variations["Variation"].astype(str).apply(normalize_name)
base_names["Normalized"] = base_names["Base_Name"].astype(str).apply(normalize_name)

### Token Sort Ratio Matching

In [31]:
def get_best_match(name, base_names, threshold=80):
    match = process.extractOne(
        name,
        base_names["Normalized"].tolist(),
        scorer=fuzz.token_sort_ratio
    )
    if match and match[1] >= threshold:
        matched_row = base_names.loc[base_names["Normalized"] == match[0], "Base_Name"].values[0]
        return matched_row, match[1]
    return None, None

results = []
for name, norm in zip(name_variations["Variation"], name_variations["Normalized"]):
    matched_name, score = get_best_match(norm, base_names)
    results.append((name, matched_name, score))

### Results DataFrame

In [32]:
matches_df = pd.DataFrame(results, columns=["Variation_Name", "Matched_Base_Name", "Score"])
matches_df.head(15)

,Variation_Name,Matched_Base_Name,Score
0,Thomas King,Thomas King,100.000000
1,ThomasKing,Thomas King,100.000000
2,Maria Garcia,Maria Garcia,100.000000
3,MaryLewis,Mary Lewis,100.000000
4,Nancy W.,None,NaN
5,Dani3l Scott,Daniel Scott,95.652174
6,JOHN smith,John Smith,100.000000
7,linda johnson,Linda Johnson,100.000000
8,N@ncy Wright,Nancy Wright,95.652174
9,William Davis,William Davis,100.000000


In [33]:
from sklearn.metrics import accuracy_score

y_true = name_variations['Matches_With_Base_Name'].fillna("No Match")
y_pred = matches_df['Matched_Base_Name'].fillna("No Match")

accuracy_score(y_true, y_pred)

0.95